In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!mkdir "/content/images"
!mkdir "/content/images/train"
!mkdir "/content/images/test"
!unrar x "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/train/train.rar" "/content/images/train"
!unrar x "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/test/test.rar" "/content/images/test"
!cp "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/train/annotations.json" "/content/images/train"
!cp "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/test/annotations.json" "/content/images/test"

Streaming output truncated to the last 5000 lines.
Extracting  /content/images/train/images/092386.jpg                       83%  OK 
Extracting  /content/images/train/images/092390.jpg                       83%  OK 
Extracting  /content/images/train/images/092391.jpg                       83%  OK 
Extracting  /content/images/train/images/092394.jpg                       83%  OK 
Extracting  /content/images/train/images/092399.jpg                       83%  OK 
Extracting  /content/images/train/images/092400.jpg                       83%  OK 
Extracting  /content/images/train/images/092414.jpg                       83%  OK 
Extracting  /content/images/train/images/092417.jpg                       83%  OK 
Extracting  /content/images/train/images/092419.jpg                       83%  OK 
Extracting  /content/images/train/images/092420.jpg                       83%  OK 
Extracting  /content/images/t

In [3]:
import cv2
import numpy as np
import sys
from os import listdir
from os.path import isfile, join
from google.colab.patches import cv2_imshow
from tqdm import tqdm
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import math

sys.path.append("/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/scripts/")
from data_generation import DataGeneration
from filter_cats import filtered_cats

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.utils import normalize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras import backend as K

%env SM_FRAMEWORK=tf.keras
!pip install segmentation_models
import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras
     |████████████████████████████████| 51kB 7.6MB/s 
Segmentation Models: using `tf.keras` framework.


In [5]:
SIZE_X = 256
SIZE_Y = 256
N_CATS = 16
ADD_BACKGROUND = False
N_CHANNELS = N_CATS + int(ADD_BACKGROUND)   # One more channel for background

INPUT_SHAPE = (SIZE_X, SIZE_Y, 3)
BATCH_SIZE = 16

TRAIN_IMAGES_PATH = "/content/images/train/images"
TRAIN_ANNOTATIONS_PATH = "/content/images/train/annotations.json"
TEST_IMAGES_PATH = "/content/images/test/images"
TEST_ANNOTATIONS_PATH = "/content/images/test/annotations.json"

In [6]:
def batch_generator(batchsize, images_path, annotation_path):
  print("Generator created for " + annotation_path)
  i_img = 0
  coco = COCO(annotation_path)

  categories_ids, categories_names, img_ids = filtered_cats(coco, n=N_CATS)
  np.random.shuffle(img_ids)

  images = coco.loadImgs(img_ids)
  img_paths = [img["file_name"] for img in images]

  data_gen = DataGeneration(coco, SIZE_X, SIZE_Y, categories_ids, add_background=ADD_BACKGROUND)

  while True:
    inputs = np.zeros((batchsize, SIZE_X, SIZE_Y, 3))
    outputs = np.zeros((batchsize, SIZE_X, SIZE_Y, N_CHANNELS))

    for i in range(batchsize):
      try:
        inputs[i] = data_gen.x_sample(join(images_path, img_paths[i_img]))
        outputs[i] = data_gen.y_sample(img_ids[i_img])
      except:
        i_img = (i_img + 1) % len(img_paths)
        inputs[i] = data_gen.x_sample(join(images_path, img_paths[i_img]))
        outputs[i] = data_gen.y_sample(img_ids[i_img])

      i_img = (i_img + 1) % len(img_paths)

    yield (inputs/255), outputs

In [7]:
def get_steps_per_epoch(batchsize, annotation_path):
  coco = COCO(annotation_path)
  categories_ids, categories_names, img_ids = filtered_cats(coco, n=N_CATS)

  return int(math.floor(len(img_ids) / batchsize))

In [8]:
def get_class_weights(annotation_path):
  class_weights = np.zeros(N_CHANNELS)
  coco = COCO(annotation_path)
  categories_ids, categories_names, img_ids = filtered_cats(coco, n=N_CATS)

  data_gen = DataGeneration(coco, SIZE_X, SIZE_Y, categories_ids, add_background=ADD_BACKGROUND)

  for id in img_ids:
    weights = data_gen.get_class_weights(id)
    if (~np.isnan(weights)).all():
      class_weights = np.add(class_weights, weights)
 
  class_weights = np.divide(class_weights, len(img_ids))
  return class_weights

In [9]:
def conv_block(input, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(input)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.1)(x)

  x = Conv2D(num_filters, 3, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  return x


def decoder_block(input, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x


def build_vgg19_unet(input_shape, n_classes=1):
  """ Input """
  inputs = Input(input_shape)

  """ Pre-trained VGG19 Model """
  vgg19 = VGG19(include_top=False, weights="imagenet", input_tensor=inputs)
  for layer in vgg19.layers:
    layer.trainable = False

  """ Encoder """
  s1 = vgg19.get_layer("block1_conv2").output         ## (512 x 512)
  s2 = vgg19.get_layer("block2_conv2").output         ## (256 x 256)
  s3 = vgg19.get_layer("block3_conv4").output         ## (128 x 128)
  s4 = vgg19.get_layer("block4_conv4").output         ## (64 x 64)

  """ Bridge """
  b1 = vgg19.get_layer("block5_conv4").output         ## (32 x 32)

  """ Decoder """
  d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
  d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
  d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
  d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)

  """ Output """
  outputs = Conv2D(n_classes, (1,1), padding="same", activation="softmax")(d4)

  model = Model(inputs, outputs, name="VGG19_U-Net")
  return model

In [10]:
def jaccard_coef(y_true, y_pred):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)
  return (intersection) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection)

def jaccard_loss(y_true, y_pred):
  return (1 - jaccard_coef(y_true, y_pred))

In [11]:
class_weights = np.ones(N_CHANNELS)
class_weights = np.divide(class_weights, get_class_weights(TRAIN_ANNOTATIONS_PATH))
sum = np.sum(class_weights)
class_weights = np.divide(class_weights, sum)

# class_weights = []

coco = COCO(TEST_ANNOTATIONS_PATH)
categories_ids, categories_names, img_ids = filtered_cats(coco, n=N_CATS)

if ADD_BACKGROUND:
  categories_names += ["background"]

print(dict(zip(categories_names, class_weights)))
np.sum(class_weights)

loading annotations into memory...
Done (t=1.68s)
creating index...
index created!


/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/scripts/data_generation.py:95: RuntimeWarning: invalid value encountered in true_divide
  return ret / total_pixels


loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
{'water': 0.02348432080781901, 'bread-white': 0.029602669256155838, 'salad-leaf-salad-green': 0.029401972874972496, 'tomato': 0.04941681957890603, 'butter': 0.05976591557209492, 'bread-wholemeal': 0.041174053960029273, 'coffee-with-caffeine': 0.053523662677300354, 'carrot': 0.05028303986196188, 'apple': 0.060862382723465676, 'mixed-vegetables': 0.07445970457757911, 'egg': 0.06761805003964108, 'tea': 0.079701658481245, 'rice': 0.08627239592429911, 'banana': 0.09280022511924023, 'mixed-salad-chopped-without-sauce': 0.09713706136094344, 'cucumber': 0.10449606718434651}


1.0

In [12]:
model = build_vgg19_unet(INPUT_SHAPE, N_CHANNELS)

total_loss = sm.losses.DiceLoss(per_image=True, class_weights=class_weights) + sm.losses.BinaryFocalLoss() + sm.losses.JaccardLoss(per_image=True, class_weights=class_weights)
# total_loss = [jaccard_loss]
# total_loss = 1/3 * sm.losses.DiceLoss(class_weights=class_weights, per_image=True) + 1/3 * sm.losses.JaccardLoss(class_weights=class_weights, per_image=True) + 1/3 * sm.losses.CategoricalFocalLoss()
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.IOUScore(threshold=0.5, per_image=True, name="iou_score_averaged"), sm.metrics.FScore(threshold=0.5, per_image=True)]

model.compile(optimizer='Adam', loss=total_loss, metrics=metrics)
model.summary()

80142336/80134624 [==============================] - 0s 0us/step
Model: "VGG19_U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
_______________________

In [13]:
callbacks = [
             ModelCheckpoint("/content/logs/model_checkpoints/food_segmentation.h5", verbose=1, save_best_only=True),
             EarlyStopping(patience=5, monitor="val_loss"),
             TensorBoard(log_dir="/content/logs/tensorboard_logs")
]

gen_train = batch_generator(BATCH_SIZE, TRAIN_IMAGES_PATH, TRAIN_ANNOTATIONS_PATH)
gen_val = batch_generator(BATCH_SIZE, TEST_IMAGES_PATH, TEST_ANNOTATIONS_PATH)

In [14]:
history = model.fit(gen_train,
                    verbose=1,
                    steps_per_epoch=get_steps_per_epoch(BATCH_SIZE, TRAIN_ANNOTATIONS_PATH),
                    epochs=20,
                    validation_data=gen_val,
                    validation_steps=get_steps_per_epoch(BATCH_SIZE, TEST_ANNOTATIONS_PATH),
                    #class_weight=class_weights,
                    callbacks=callbacks)

loading annotations into memory...
Done (t=1.68s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Generator created for /content/images/train/annotations.json
loading annotations into memory...
Done (t=1.62s)
creating index...
index created!
Epoch 1/20
835/835 [==============================] - ETA: 0s - loss: 2.0045 - iou_score: 0.2127 - iou_score_averaged: 0.8545 - f1-score: 0.8571Generator created for /content/images/test/annotations.json
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
835/835 [==============================] - 651s 726ms/step - loss: 2.0045 - iou_score: 0.2127 - iou_score_averaged: 0.8545 - f1-score: 0.8571 - val_loss: 2.0131 - val_iou_score: 0.1848 - val_iou_score_averaged: 0.8263 - val_f1-score: 0.8295

Epoch 00001: val_loss improved from inf to 2.01312, saving model to /content/logs/model_checkpoints/food_segmentation.h5
Epoch 2/20
835/835 [====================

In [15]:
!cp "/content/logs/model_checkpoints/food_segmentation.h5" "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/weights/vgg19_pretrained_facu.h5"

In [16]:
!cp -r "/content/logs/tensorboard_logs" "/c ontent/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/tensorboard_logs/VGG19"

cp: cannot create directory '/c ontent/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/tensorboard_logs/VGG19': No such file or directory
